In [5]:
import pandas as pd
from Basilik import BN, Node

In [6]:
# define nodes/edges

C = Node("cloudy", None)
R = Node("rain", [C])
S = Node("sprinkler", [C])
W = Node("wet", [R, S])

ls_n = [C, R, S, W]

In [7]:
# fetch joint observations

obs = pd.read_csv("observations.csv").drop("Unnamed: 0", axis=1)
obs.head()

,cloudy,rain,sprinkler,wet
0,True,True,False,True
1,False,False,False,False
2,False,True,False,True
3,False,False,False,False
4,False,False,False,False


In [8]:
# construct model with nodes and observations

model = BN(ls_n, obs)

In [11]:
# easily view conditional probabilities for each node
model.generate_cpt("sprinkler")

sprinkler,cloudy,False,True
0,False,0.496855,0.503145
1,True,0.892925,0.107075


In [ ]:
# easily view conditional probabilities for each node
model.generate_cpt("wet")